In [ ]:
# This is not production grade code.  
# I published it so that I could share the code, ask for help, 
# and get feedback from the community.

In [ ]:
import pandas as pd
from nautilus_trader.persistence.wranglers import BarDataWrangler
from nautilus_trader.persistence.loaders import CSVBarDataLoader
from nautilus_trader.test_kit.providers import TestInstrumentProvider
from nautilus_trader.backtest.engine import BacktestEngine
from nautilus_trader.backtest.engine import BacktestEngineConfig
from nautilus_trader.model.data import BarType
from nautilus_trader.model.identifiers import Venue
from nautilus_trader.model.currencies import USD
from nautilus_trader.model.enums import AccountType
from nautilus_trader.model.enums import OmsType
from nautilus_trader.model.objects import Money
from nautilus_trader.trading.strategy import Strategy
from nautilus_trader.model.enums import OrderSide
from nautilus_trader.config import LoggingConfig
from nautilus_trader.config import BacktestRunConfig
from nautilus_trader.backtest.node import BacktestNode
from nautilus_trader.backtest.results import BacktestResult

In [ ]:
# process data

data_path = "./data/EURUSD.csv"
df = CSVBarDataLoader.load(data_path)
instrument = TestInstrumentProvider.default_fx_ccy("EUR/USD")
bar_type = BarType.from_str("EUR/USD.SIM-1-DAY-LAST-EXTERNAL")
wrangler = BarDataWrangler(
  bar_type=bar_type,
  instrument=instrument
)

bars = wrangler.process(df)

# Configure backtest engine
config = BacktestEngineConfig(trader_id="BACKTESTER-001")

# Build the backtest engine
engine = BacktestEngine(config=config)

# add venue

SIM = Venue("SIM")

engine.add_venue(
  venue=SIM,
  oms_type=OmsType.NETTING,  
  account_type=AccountType.MARGIN,
  base_currency=USD,  
  starting_balances=[Money(100_000, USD)], 
  bar_execution=True
)

# add data to engine

engine.add_instrument(instrument)
engine.add_data(bars)

In [ ]:
# strategy

class BuyAndHold(Strategy):
  def __init__(self) -> None:
    super().__init__() 

  def on_start(self) -> None:
    self.instrument = instrument
    self.bar_type = bar_type
    
    if self.instrument is None:
      self.log.error(f"Could not find instrument for {self.instrument_id}")
      self.stop()
      return
    
    self.request_instrument(self.instrument)
    
    try:
      self.request_bars(self.bar_type)
    except:
      self.log.error(f"Could not find bars for {self.bar_type}")
      self.stop()
      return
    
  def on_stop(self) -> None:
    self.close_all_positions(self.instrument_id)

  def on_bar(self) -> None:
    try:
      self.market_order(
        instrument_id=self.instrument_id,
        order_side=OrderSide.BUY,
        venue_id=SIM,
        quantity=1
      )
    except:
      self.log.error("An order was not placed.")

In [ ]:
# backtest engine config and logging

engine = BacktestEngineConfig(
  trader_id="TESTER-001",
  logging=LoggingConfig(
    log_level="INFO",
    log_level_file="DEBUG",
    log_file_format="json",
    log_component_levels={ "Portfolio": "INFO" },
  ),
)

In [ ]:
# run engine

engine.run()

  # Optionally view reports
with pd.option_context(
    "display.max_rows",
    100,
    "display.max_columns",
    None,
    "display.width",
    300,
):
  print(engine.trader.generate_account_report(SIM))
  print(engine.trader.generate_order_fills_report())
  print(engine.trader.generate_positions_report())

# For repeated backtest runs make sure to reset the engine
engine.reset()

# Good practice to dispose of the object when done
engine.dispose()